In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import pandas as pd
# QA
inputs = [
    "For customer-facing applications, which company's models dominate the top rankings?",
    "What percentage of respondents are using RAG in some form?",
    "How often are most respondents updating their models?",
]

outputs = [
    "OpenAI models dominate, with 3 of the top 5 and half of the top 10 most popular models for customer-facing apps.",
    "70% of respondents are using RAG in some form.",
    "More than 50% update their models at least monthly, with 17% doing so weekly.",
]

# Dataset
qa_pairs = [{"question": q, "answer": a} for q, a in zip(inputs, outputs)]
df = pd.DataFrame(qa_pairs)

# write to csv
csv_path = "D:\LLMOPs\Multi-Chat-Doc\data\goldens.csv"
df.to_csv(csv_path, index=False)

<>:20: SyntaxWarning: invalid escape sequence '\L'
<>:20: SyntaxWarning: invalid escape sequence '\L'
C:\Users\user\AppData\Local\Temp\ipykernel_18916\610711822.py:20: SyntaxWarning: invalid escape sequence '\L'
  csv_path = "D:\LLMOPs\Multi-Chat-Doc\data\goldens.csv"


In [ ]:
from langsmith import Client

client = Client()
dataset_name = "llmops_dataset"

# Store
dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Input and expected output for the AgenticAI report.",
)
client.create_examples(
    inputs=[{"question": q} for q in inputs],
    outputs=[{"answer": a} for a in outputs],
    dataset_id=dataset.id,
)

{'example_ids': ['05365b30-c459-4f93-9ae0-7dd93830e3ff',
  '30bba0de-984e-4fdb-bcfb-3e70e81e2920',
  'bf165f72-78c3-4455-8fb5-59751637f711'],
 'count': 3}

In [ ]:
import sys
# sys.path.append("")

from pathlib import Path
from multi_doc_chat.src.document_ingestion.data_ingestion import ChatIngestor
from multi_doc_chat.src.document_chat.retrieval import ConversationalRAG
import os

# Simple file adapter for local file paths
class LocalFileAdapter:
    """Adapter for local file paths to work with ChatIngestor."""
    def __init__(self, file_path: str):
        self.path = Path(file_path)
        self.name = self.path.name
    
    def getbuffer(self) -> bytes:
        return self.path.read_bytes()


def answer_ai_report_question(
    inputs: dict,
    data_path: str = "D:\LLMOPs\Multi-Chat-Doc\data\The 2025 AI Engineering Report.pdf",
    chunk_size: int = 1000,
    chunk_overlap: int = 200,
    k: int = 5
) -> dict:
    """
    Answer questions about the AI Engineering Report using RAG.
    
    Args:
        inputs: Dictionary containing the question, e.g., {"question": "What is RAG?"}
        data_path: Path to the AI Engineering Report text file
        chunk_size: Size of text chunks for splitting
        chunk_overlap: Overlap between chunks
        k: Number of documents to retrieve
    
    Returns:
        Dictionary with the answer, e.g., {"answer": "RAG stands for..."}
    """
    try:
        # Extract question from inputs
        question = inputs.get("question", "")
        if not question:
            return {"answer": "No question provided."}

        # Check if file exists
        if not Path(data_path).exists():
            return {"answer": f"File not found: {data_path}"}

        # Create file adapter
        file_adapter = LocalFileAdapter(data_path)

        # Build index using ChatIngestor
        ingestor = ChatIngestor(
            temp_base = "data",
            faiss_base = "faiss_index",
            use_session_dirs=True
        )

        # Build retriever
        ingestor.built_retriver(
            uploaded_files=[file_adapter],
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            k=k
        )

        # Get session ID and Index path
        session_id = ingestor.session_id
        index_path = f"faiss_index/{session_id}"

        # Create RAG instance and load retriver
        rag = ConversationalRAG(session_id=session_id)
        rag.load_retriever_from_faiss(
            index_path=index_path,
            k=k,
            index_name=os.getenv("FAISS_INDEX_NAME", "index")
        )

        # Get answer
        answer = rag.invoke(question, chat_history=[])

        return {"answer": answer}
    except Exception as e:
        return {"answer": f"An error occurred: {str(e)}"}
    

In [ ]:
# Test the function with a sample question
import test


test_input = {"question": "For customer-facing applications, which company's modell dominate the top rankings"}
result = answer_ai_report_question(test_input)
print("Question:", test_input["question"])
print("\nAnswer:", result["answer"])

In [1]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator

In [ ]:
# Example: Test with all gol[den questions
print("Testing all questions from the dataset:\n")
for i, q in enumerate(inputs, 1):
    test_input = {"question": q}
    result = answer_ai_report_question(test_input)
    print(f"Q{i}: {q}")
    print(f"A{i}: {result['answer']}\n")
    print("*" * 80 + "\n")

#### Implementing LangSmith in-built evaluation

In [ ]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator

# Evaluators
qa_evaluator = [LangChainStringEvaluator("cot_qa")]
dataset_name = "llmops_dataset"

# Run evaluation using our RAG function
experiment_results = evaluate(
    answer_ai_report_question,
    data=dataset_name,
    evaluators=qa_evaluator,
    experiment_prefix="test-llmops_dataset-qa-rag",
    # Experiment metadata
    metadata = {
        "variant": "RAG with FAISS and AI Engineering Report",
        "chunk_size": 1000,
        "chunk_overlap": 200,
        "k": 5,
    },
)